In [1]:
import pandas as pd
from nltk import word_tokenize

In [2]:
df = pd.read_csv('poco_f1_edited.csv')

In [3]:
df.head()

,review_name,review_rating
0,Such a wonderful phone in this range\n845 snap...,5
1,Expected a replacement and had battery issue b...,2
2,Received the phone was happy to see the packag...,5
3,Its all what i wanted from a phone.The fingerp...,5
4,Beast at this price range,5


In [4]:
for index in range(len(df['review_rating'])):
    val = df['review_rating'][index]
   # print(val)
    try:
        df['review_rating'][index] = int(val)
        
    except ValueError:
        df['review_rating'][index] = 3
        #print(val)
    

In [5]:
s = type(df['review_rating'][0])
for index,row in df.iterrows():
    #print(type(row['review_rating']))
    if type(row['review_rating'])  != s:
        df['review_rating'][index]= 3

In [6]:

positive_list = []
negative_list = []
for i in range(11862):
    if(df.review_rating[i]<3):
        negative_list.append(df.review_name[i])
    else:
        positive_list.append(df.review_name[i])

In [7]:
import nltk

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

In [9]:
import re, string, random

In [10]:
for i in range(11862):
    df.review_rating[i] =int(df.review_rating[i])

In [11]:
p_list =[]
n_list=[]
for i in range(len(positive_list)):
    p_list.append(word_tokenize(positive_list[i]))
for i in range(len(negative_list)):
    n_list.append(word_tokenize(negative_list[i]))

In [12]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [13]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [14]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [15]:
print(positive_list[0])

Such a wonderful phone in this range
845 snapdragon which is the latest that is in it.
Just go for it.
Sexy phone


In [16]:
if __name__ == "__main__":

    stop_words = stopwords.words('english')

    positive_tweet_tokens = p_list
    negative_tweet_tokens = n_list

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    print(classifier.show_most_informative_features(10))

    custom_tweet = "this camera is bullshit"

    custom_tokens = remove_noise(word_tokenize(custom_tweet))

    print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[('phone', 7672), ('good', 6031), ('camera', 3825), ('...', 2986), ('best', 2796), ('battery', 2673), ('performance', 2601), ('..', 2190), ('price', 2174), ('use', 1772)]
Accuracy is: 0.8932538050185109
Most Informative Features
                  reject = True           Negati : Positi =     36.2 : 1.0
             motherboard = True           Negati : Positi =     33.1 : 1.0
                   10-15 = True           Negati : Positi =     28.9 : 1.0
               executive = True           Negati : Positi =     28.9 : 1.0
                manually = True           Negati : Positi =     28.9 : 1.0
                replaced = True           Negati : Positi =     28.9 : 1.0
                   swell = True           Negati : Positi =     28.9 : 1.0
               defective = True           Negati : Positi =     27.3 : 1.0
                 compass = True           Negati : Positi =     23.6 : 1.0
                  sudden = True           Negati : Positi =     23.6 : 1.0
None
this camera is b

In [17]:
print(positive_cleaned_tokens_list[0])

['wonderful', 'phone', 'range', '845', 'snapdragon', 'late', 'go', 'sexy', 'phone']


In [18]:
import nltk

In [19]:
import spacy

In [20]:
nlp = spacy.load("en_core_web_sm")

In [21]:
sentences = [
  'The food we had yesterday was delicious',
  'My time in Italy was very enjoyable',
  'I found the meal to be tasty',
  'The internet was slow.',
  'Our experience was suboptimal',
  'the camera is not great'
]

In [22]:
for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
              token.pos_,[child for child in token.children])

The det food NOUN DET []
food nsubj was AUX NOUN [The, had]
we nsubj had VERB PRON []
had relcl food NOUN VERB [we, yesterday]
yesterday npadvmod had VERB NOUN []
was ROOT was AUX AUX [food, delicious]
delicious acomp was AUX ADJ []
My poss time NOUN PRON []
time nsubj was AUX NOUN [My, in]
in prep time NOUN ADP [Italy]
Italy pobj in ADP PROPN []
was ROOT was AUX AUX [time, enjoyable]
very advmod enjoyable ADJ ADV []
enjoyable acomp was AUX ADJ [very]
I nsubj found VERB PRON []
found ROOT found VERB VERB [I, be]
the det meal NOUN DET []
meal nsubj be VERB NOUN [the]
to aux be VERB PART []
be ccomp found VERB VERB [meal, to, tasty]
tasty acomp be VERB ADJ []
The det internet NOUN DET []
internet nsubj was AUX NOUN [The]
was ROOT was AUX AUX [internet, slow, .]
slow acomp was AUX ADJ []
. punct was AUX PUNCT []
Our poss experience NOUN PRON []
experience nsubj was AUX NOUN [Our]
was ROOT was AUX AUX [experience, suboptimal]
suboptimal acomp was AUX ADJ []
the det camera NOUN DET []
camer

In [23]:
for sentence in sentences:
    doc = nlp(sentence)
    descriptive_term = ''
    for token in doc:
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = prepend + token.text
print(sentence)
print(descriptive_term)

the camera is not great
great


In [24]:
aspects = []
for sentence in sentences:
    doc = nlp(sentence)
    descriptive_term = ''
    target = ''
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            target = token.text
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
                descriptive_term = prepend + token.text
    aspects.append({'aspect': target,
    'description': descriptive_term})
print(aspects)

[{'aspect': 'food', 'description': ''}, {'aspect': 'time', 'description': 'very enjoyable'}, {'aspect': 'meal', 'description': ''}, {'aspect': 'internet', 'description': ''}, {'aspect': 'experience', 'description': ''}, {'aspect': 'camera', 'description': ''}]


In [25]:
from textblob import TextBlob
for aspect in aspects:
    aspect['sentiment'] = TextBlob(aspect['description']).sentiment
print(aspects)

[{'aspect': 'food', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'time', 'description': 'very enjoyable', 'sentiment': Sentiment(polarity=0.65, subjectivity=0.78)}, {'aspect': 'meal', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'internet', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'experience', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'camera', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}]


In [26]:
from textblob.classifiers import NaiveBayesClassifier
# We train the NaivesBayesClassifier
train = [
  ('Slow internet.', 'negative'),
  ('Delicious food', 'positive'),
  ('Suboptimal experience', 'negative'),
  ('Very enjoyable time', 'positive'),
  ('delicious food.', 'neg')
]
cl = NaiveBayesClassifier(train)
# And then we try to classify some sample sentences.
blob = TextBlob("Delicious food. Very Slow internet. Suboptimal experience. Enjoyable food.", classifier=cl)
for s in blob.sentences:
    print(s)
    print(s.classify())

Delicious food.
positive
Very Slow internet.
negative
Suboptimal experience.
negative
Enjoyable food.
positive


In [27]:
cleaned_data_pos = []
for line in positive_list:
    cleaned_data_pos.append(remove_noise(tokens,stop_words))

In [28]:
cleaned_data_neg = []
for line in negative_list:
    cleaned_data_neg.append(remove_noise(tokens,stop_words))

In [29]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [30]:
sentence_pos = []
for line in cleaned_data_pos:
    sentence_pos.append(TreebankWordDetokenizer().detokenize(line))


In [31]:
sentence_pos[0]

'local gui'

In [32]:
sentence_neg = []
for line in cleaned_data_neg:
    sentence_neg.append(TreebankWordDetokenizer().detokenize(line))

In [33]:
sentence_neg[0]

'local gui'

In [34]:
p_list = []
n_list=[]
for i in range(len(positive_list)):
    p_list.append(word_tokenize(positive_list[i]))
for i in range(len(negative_list)):
    n_list.append(word_tokenize(negative_list[i]))

In [35]:
positive_token = p_list
positive_cleaned =[]

In [36]:
for tokens in positive_token:
    positive_cleaned.append(remove_noise(tokens,stop_words))

In [37]:
all_pos_words = get_all_words(positive_cleaned)
positive_tokens_for_model = get_tweets_for_model(positive_cleaned)


In [38]:
positive_cleaned[0]

['wonderful',
 'phone',
 'range',
 '845',
 'snapdragon',
 'late',
 'go',
 'sexy',
 'phone']

In [39]:
for i in range(len(positive_cleaned)):
    positive_cleaned[i]=TreebankWordDetokenizer().detokenize(positive_cleaned[i])

In [40]:
negative_token = n_list
negative_cleaned=[]


In [41]:
for tokens in negative_token:
    negative_cleaned.append(remove_noise(tokens, stop_words))

In [42]:
for i in range(len(negative_cleaned)):
    negative_cleaned[i]=TreebankWordDetokenizer().detokenize(negative_cleaned[i])

In [43]:
review_list = positive_cleaned + negative_cleaned

In [44]:
sentences = review_list[:5]

In [45]:
for sentence in sentences:
  doc = nlp(sentence)
  for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
      token.pos_,[child for child in token.children])

wonderful amod range NOUN ADJ []
phone compound range NOUN NOUN []
range nsubj go VERB NOUN [wonderful, phone, snapdragon]
845 nummod snapdragon NOUN NUM []
snapdragon appos range NOUN NOUN [845]
late advmod go VERB ADV []
go ROOT go VERB VERB [range, late, phone]
sexy amod phone NOUN ADJ []
phone dobj go VERB NOUN [sexy]
received ROOT received VERB VERB [happy, see, 've, need, perfectly]
phone compound happy ADJ NOUN []
happy dobj received VERB ADJ [phone]
see advcl received VERB VERB [guard, back, cover]
packaging nsubj get NOUN NOUN []
get compound guard NOUN NOUN [packaging]
screen compound guard NOUN NOUN []
guard dobj see VERB NOUN [get, screen]
back advmod see VERB ADV []
cover conj see VERB NOUN [guard]
early amod guard NOUN ADJ []
get compound screen NOUN NOUN []
screen compound guard NOUN NOUN [get]
guard dobj cover NOUN NOUN [early, screen, layern't, perfect]
early amod scratch NOUN ADJ []
anti amod scratch NOUN ADJ []
scratch compound layern't PROPN NOUN [early, anti]
layer

In [46]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      descriptive_term = token
  print(sentence)
  print(descriptive_term)

wonderful phone range 845 snapdragon late go sexy phone
sexy
received phone happy see packaging get screen guard back cover early get screen guard early anti scratch layern't last long 3rd generation gorilla glass phone perfect although build quality could've good happy one cater need perfectly
happy
want phone.the fingerprint scanner fast.the ir face unlock fast.and performance also great.overall great phone
great
beast price range

amazing pic n video...love phone .. nice n fast .. awesome pic great battery life .. real flagship phone
real


In [47]:
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  for token in doc:
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  print(sentence)
  print(descriptive_term)

wonderful phone range 845 snapdragon late go sexy phone
sexy
received phone happy see packaging get screen guard back cover early get screen guard early anti scratch layern't last long 3rd generation gorilla glass phone perfect although build quality could've good happy one cater need perfectly
happy
want phone.the fingerprint scanner fast.the ir face unlock fast.and performance also great.overall great phone
great
beast price range

amazing pic n video...love phone .. nice n fast .. awesome pic great battery life .. real flagship phone
real


In [48]:
aspects = []
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  target = ''
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  aspects.append({'aspect': target,
    'description': descriptive_term})
print(aspects)



[{'aspect': 'range', 'description': 'sexy'}, {'aspect': 'quality', 'description': 'happy'}, {'aspect': 'scanner', 'description': 'great'}, {'aspect': '', 'description': ''}, {'aspect': 'pic', 'description': 'real'}]


In [49]:
from textblob import TextBlob
for aspect in aspects:
  aspect['sentiment'] = TextBlob(aspect['description']).sentiment
print(aspects)

[{'aspect': 'range', 'description': 'sexy', 'sentiment': Sentiment(polarity=0.5, subjectivity=1.0)}, {'aspect': 'quality', 'description': 'happy', 'sentiment': Sentiment(polarity=0.8, subjectivity=1.0)}, {'aspect': 'scanner', 'description': 'great', 'sentiment': Sentiment(polarity=0.8, subjectivity=0.75)}, {'aspect': '', 'description': '', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'pic', 'description': 'real', 'sentiment': Sentiment(polarity=0.2, subjectivity=0.30000000000000004)}]


In [50]:
##np.save
##np.load


In [51]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(sentence)

for token in doc:
    print(token.text , token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

amazing amazing ADJ JJ amod xxxx True False
pic pic NOUN NN nsubj xxx True False
n n CCONJ CC cc x True False
video video NOUN NN conj xxxx True False
... ... PUNCT : punct ... False False
love love NOUN NN compound xxxx True False
phone phone NOUN NN ROOT xxxx True False
.. .. PUNCT NFP punct .. False False
nice nice ADJ JJ amod xxxx True False
n n CCONJ CC cc x True False
fast fast ADJ JJ conj xxxx True False
.. .. PUNCT NFP punct .. False False
awesome awesome ADJ JJ amod xxxx True False
pic pic NOUN NN nmod xxx True False
great great ADJ JJ amod xxxx True False
battery battery NOUN NN compound xxxx True False
life life NOUN NN appos xxxx True False
.. .. PUNCT NFP punct .. False False
real real ADJ JJ amod xxxx True False
flagship flagship NOUN NN compound xxxx True False
phone phone NOUN NN appos xxxx True False


In [52]:
features = ['battery','ram','display','camera','storage']

In [53]:
from textblob import TextBlob

In [54]:
blob = TextBlob(sentence)

In [55]:
print(blob)
blob.sentiment

amazing pic n video...love phone .. nice n fast .. awesome pic great battery life .. real flagship phone


Sentiment(polarity=0.5666666666666668, subjectivity=0.7583333333333333)

In [56]:
blob.correct()

TextBlob("amazing pick n video...love phone .. nice n fast .. awesome pick great battery life .. real flagship phone")

In [58]:
import numpy as np
np.save('preprocessed_array_sentence',sentence)

In [62]:
reviews = np.load('preprocessed_array_sentence.npy')

In [63]:
reviews[0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed